In [1]:
!pip list

[Executed on 2023-09-07 12:00:44.940634]
Package                           Version
--------------------------------- -------------
absl-py                           1.4.0
anndata                           0.9.2
annotated-types                   0.5.0
annoy                             1.17.2
anyio                             3.5.0
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
asttokens                         2.2.1
attrs                             22.1.0
backcall                          0.2.0
backoff                           2.2.1
backports.functools-lru-cache     1.6.5
beautifulsoup4                    4.12.2
bleach                            4.1.0
blessed                           1.19.1
boltons                           23.0.0
Brotli                            1.0.9
build                             0.10.0
CacheControl                      0.13.1
cached-property                   1.5.2
certifi             

In [5]:
!mamba install rpy2 r-essentials -c conda-forge -y

[Executed on 2023-09-07 12:03:01.147022]

Looking for: ['rpy2', 'r-essentials']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
pkgs/main/noarch                                              No change
[+] 0.1s
pkgs/main/linux-64 ━━━━━━━━━━━━━━━╸━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/linux-64    ━━━━━━╸━━━━━━━━━━━━━━━╸━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/noarch      ━━━━━━━╸━━━━━━━━━━━━━━━╸━   0.0 B /  ??.?MB @  ??.?MB/s  0.1spkgs/r/linux-64                                               No change
pkgs/r/noarch                                                 No change
[+] 0.2s
pkgs/main/linux-64 ━━━━━━━━━╸━━━━━━━━━━━━━━━   2.2MB /   5.3MB @  13.5MB/s  0.2spkgs/main/linux-64                                   5.3MB @  23.7MB/s  0.3s

Pinned packages:
  - python 3.9.*


warning  libmamba Extracted package cache '/home/juany/.conda/pkgs/fontconfig-2.14.2-h14ed4e7_0'

In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.packages import STAP

def zumis_output2pandas(filename):
    mfunc = 'to_df <- function(dobj){return(as.data.frame(as.matrix(dobj)))}'
    rsparse2pandas = STAP(mfunc, "to_df")

    readRDS = robjects.r['readRDS']
    zumis_data = readRDS(filename)
    zd = dict(zip(zumis_data.names, list(zumis_data)))
    for key in ['umicount', 'readcount']:
        zd[key] = dict(zip(zd[key].names, list(zd[key])))
        
        for sub_key in ['exon', 'inex', 'intron']:
            zd[key][sub_key] = dict(zip(zd[key][sub_key].names, list(zd[key][sub_key])))
            zd[key][sub_key]['all'] = rsparse2pandas.to_df(zd[key][sub_key]['all'])
            zd[key][sub_key]['all'] = robjects.conversion.get_conversion().rpy2py(zd[key][sub_key]['all']).T
            zd[key][sub_key]['downsampling'] = dict(zip(zd[key][sub_key]['downsampling'].names, list(zd[key][sub_key]['downsampling'])))
            
            for k in zd[key][sub_key]['downsampling'].keys():
                zd[key][sub_key]['downsampling'][k] = rsparse2pandas.to_df(zd[key][sub_key]['downsampling'][k])
                zd[key][sub_key]['downsampling'][k] = robjects.conversion.get_conversion().rpy2py(zd[key][sub_key]['downsampling'][k]).T
    return zd


[Executed on 2023-09-07 12:06:26.819649]


In [2]:
a=zumis_output2pandas("/proj/sensoryaging/Xpress.dgecounts.rds")

[Executed on 2023-09-07 12:06:31.128893]


R[write to console]: Loading required package: Matrix



In [3]:
a

[Executed on 2023-09-07 12:43:53.132123]


{'umicount': {'exon': {'all':                       ENSMUSG00000000001.5  ENSMUSG00000000003.16  \
   AACCACATCGAACGCCTCGA                   0.0                    0.0   
   AACCACATCGAAGACCGTAA                   0.0                    0.0   
   AACCACATCGAAGCTGACCG                   0.0                    0.0   
   AACCACATCGACAACTCGAA                   0.0                    0.0   
   AACCACATCGACCATATGAG                   2.0                    0.0   
   ...                                    ...                    ...   
   TTGATCTTGGTGCGGCAGTA                   0.0                    0.0   
   TTGATCTTGGTGGTTCGCAA                   0.0                    0.0   
   TTGATCTTGGTTGCCTCGAT                   0.0                    0.0   
   TTGATCTTGGTTGCGCTAAT                   0.0                    0.0   
   TTGATCTTGGTTGGCTGTTG                   0.0                    0.0   
   
                         ENSMUSG00000000028.16  ENSMUSG00000000031.17  \
   AACCACATCGAACGCCTCGA         

In [6]:
a['umicount'].keys()

[Executed on 2023-09-07 13:09:46.897624]


dict_keys(['exon', 'inex', 'intron'])

In [9]:
import os 
# 定义文件夹路径
folder_path = "/proj/sensoryaging/data/data/up_dated_data/SGNs/zumi_output"

# 遍历 a['umicount'] 的键
for key in a['umicount'].keys():
    # 构建文件名
    filename = f"zumis_umicount_{key}_all.csv"
    
    # 完整的文件路径
    file_path = os.path.join(folder_path, filename)
    
    # 保存到 CSV 文件
    a['umicount'][key]['all'].to_csv(file_path)


[Executed on 2023-09-07 13:14:02.655695]
